<a href="https://colab.research.google.com/github/brianxiadong/LLMs-from-scratch-CN-Plus/blob/main/ch02/01_main-chapter-code/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章: 处理文本数据

本notebook会使用到的python包：


In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu124
tiktoken version: 0.9.0


- 本章主要包含数据准备和采样，准备好数据提供给LLM大语言模型使用

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Understanding word embeddings

- 本节不包含代码

- embeddings嵌入有多重不同的形式，比如视频、音频、文本，本书中只关注文本内容。

如下图所示，embedding嵌入模型的主要作用就是将不同格式的数据，转换为计算机能理解的嵌入向量数据。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMs 在高维空间（甚至可以达到数千维）中会与embeddings嵌入一起协同工作
- 一般人类无法可视化如此高维的空间（我们只在 1、2 或 3 维中思考），下图展示了一个二维嵌入空间

通过下图也能看到 鸭、鹅在这个二位空间中，是比较相近的，他们被圈了起来。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本分词

- 在这一部分，我们将文本进行分词，即将文本拆分成更小的单元，例如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们想要处理的原始文本
- 这里我们使用到的数据是伊迪丝·华顿所著的《判决》，这是一部公共领域的短篇小说

In [3]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

- 如果执行上一个代码单元时遇到 ssl.SSLCertVerificationError ，可能是由于使用了过时的 Python 版本；这里我建议你使用[Google Colab](https://colab.research.google.com/)


In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 接下来我们的目标是将这段文本进行分词，并生成嵌入向量数据交给LLM
- 我们一起来开发一个简单的分词器，将英文中单词进行分割
- 下面我们使用一个正则，按照空格来进行分割

> re.split(r'(\s)', text) 是 Python 中使用正则表达式分割字符串的操作，其作用是：按照空白字符（空格、制表符 \t、换行符 \n 等）分割字符串，并保留分割时使用的空白字符。


In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 这里显然有点问题，我们不仅想按空格分割，还想按逗号和句号分割，所以让我们修改正则表达式

In [6]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 这里有一些空的字符串，先把他们去掉

> 这段代码的作用是过滤列表中仅包含空白字符（或为空）的元素，并打印过滤后的结果，具体解析如下：
- 列表推导式：[item for item in result if item.strip()]
这是一个 Python 列表推导式，用于对原列表result进行过滤，生成新列表。
for item in result：遍历原列表result中的每个元素item。
if item.strip()：过滤条件。item.strip()是字符串方法，作用是移除item前后的空白字符（包括空格、换行符\n、制表符\t等）。
如果item是仅由空白字符组成的字符串（如" "、"\t\n"）或空字符串""，item.strip()会返回空字符串""，在 Python 中被视为False，因此这类元素会被过滤掉。
如果item包含非空白字符（如"hello"、" world "），item.strip()返回非空字符串，被视为True，因此这类元素会被保留。

In [7]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 我们也处理其他类型的标点符号，比如句号、问号等等

In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 现在可以应用这种分词方法到原始文本上

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算总的分词数量

In [10]:
print(len(preprocessed))

4690


## 2.3 将分词转换为分词 ID

- 接下来，我们将文本分词转换为分词 ID，以便稍后通过嵌入层进行处理

> - （一）步骤 1：标记化（Tokenization）
标记化是将连续文本分解为离散单元（称为 “标记”，Token）的过程。
输入：图中的 “完整训练数据集” 仅包含一个示例句子 “The quick brown fox jumps over the lazy dog”（译文：“那只敏捷的棕色狐狸跳过了懒狗”），该简化设定仅为便于演示。
输出：经过标记化后，文本被拆分为单个标记，形成 “已标记化的训练数据集”（如 The、quick、brown、fox、jumps、over、the、lazy、dog）。
- （二）步骤 2：词汇表构建与标记 ID 映射
  词汇表是训练集中所有唯一标记的集合，构建时遵循以下规则：
  收集唯一标记：从标记化后的结果中筛选出不重复的标记（如示例中，the 出现两次，但仅算一个唯一标记）。
  按字母排序：唯一标记被按字母顺序排列（示例中排序后为 brown、dog、fox、jumps、lazy、over、quick、the）。
  分配标记 ID：每个唯一标记被映射到一个唯一的整数（称为 “标记 ID”，Token ID），作为计算机可处理的数字表示。





<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从上面那我们获得的token中（每个词）, 我们现在可以构建一个包含所有唯一 token 的词汇表

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [23]:
vocab = {token:integer for integer,token in enumerate(all_words)}
print(vocab)

{'!': 0, '"': 1, "'": 2, '(': 3, ')': 4, ',': 5, '--': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'Ah': 12, 'Among': 13, 'And': 14, 'Are': 15, 'Arrt': 16, 'As': 17, 'At': 18, 'Be': 19, 'Begin': 20, 'Burlington': 21, 'But': 22, 'By': 23, 'Carlo': 24, 'Chicago': 25, 'Claude': 26, 'Come': 27, 'Croft': 28, 'Destroyed': 29, 'Devonshire': 30, 'Don': 31, 'Dubarry': 32, 'Emperors': 33, 'Florence': 34, 'For': 35, 'Gallery': 36, 'Gideon': 37, 'Gisburn': 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, 'His': 51, 'How': 52, 'I': 53, 'If': 54, 'In': 55, 'It': 56, 'Jack': 57, 'Jove': 58, 'Just': 59, 'Lord': 60, 'Made': 61, 'Miss': 62, 'Money': 63, 'Monte': 64, 'Moon-dancers': 65, 'Mr': 66, 'Mrs': 67, 'My': 68, 'Never': 69, 'No': 70, 'Now': 71, 'Nutley': 72, 'Of': 73, 'Oh': 74, 'On': 75, 'Once': 76, 'Only': 77, 'Or': 78, 'Perhaps': 79, 'Poor': 80, 'Professional': 81, 'Renaissance': 82, 'Ri

- 以下是这份词汇表的前 50 个词条：

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 接下来，我们使用一个小词汇表来展示一个简短样本文本的标记化过程：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容整合到一个分词器类中，使用python中的class封装一下。

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 函数将文本转换为 token ID
- `decode` 函数将 token ID 还原为文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）
- 这些整数随后可以被嵌入（一会儿实现）作为 LLM 的输入

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 通过`decode`函数，我们可以将整数还原为文本

In [16]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文token

- 在处理过程中，为了表示文本结束，我们需要添加一些特殊的token

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊token来帮助 LLM 提供额外上下文
- 这些特殊标记包括 :
    `[BOS]` (序列开始) 标记文本的开始

    `[EOS]` (序列结束) 标记文本结束的位置（通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书等）

    `[PAD]` (填充) 如果我们用大于 1 的批量大小训练 LLMs（我们可能包含多个不同长度的文本；使用填充标记我们将较短的文本填充到最长长度，以便所有文本具有相同的长度）
    
     `[UNK]` 表示不在词汇表中的单词

- GPT-2 不需要上述任何标记，只使用一个 <|endoftext|> 标记来降低复杂性
- <|endoftext|> 与上述提到的 [EOS] 标记类似, 标记文本结束的位置
- GPT 也使用 <|endoftext|> 进行填充（由于我们在批量输入训练时通常使用掩码，因此无论如何都不会关注填充的标记，所以这些标记是什么并不重要）
- GPT-2 不使用 <UNK> 标记来表示词汇表外的单词；相反，GPT-2 使用字节对编码（BPE）分词器，该分词器将单词分解为子词单元，我们将在后面的章节中讨论


- 添加 <|endoftext|> 标记在两个独立的文本源之间 ：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看如果我们对以下文本进行分词会发生什么：

In [19]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上面的代码会直接报错，因为词汇表中不包含单词"Hello"
- 为了处理这种情况，我们可以向词汇表中添加特殊标记如 "<|unk|>" 来表示未知单词
- 既然我们已经扩展了词汇表，让我们再添加一个名为 "<|endoftext|>" 的标记，它在 GPT-2 训练中用于表示文本的结束（如果我们的训练数据集由多个文章、书籍等组成，也可以用于连接的文本之间）

In [25]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [26]:
len(vocab.items())

1132

In [27]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 接下来需要相应地调整分词器，这样才能对新的单词使用 `unk`标记

In [28]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们尝试使用修改后的分词器对文本进行分词：:

In [29]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [30]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [31]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair 编码

- GPT-2 使用 BytePair Encoding（BPE）作为其分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元，甚至单个字符，从而使其能够处理词汇表之外的单词。
- 例如，如果 GPT-2 的词汇表中没有"unfamiliarword"这个词，它可能会将其分词为["unfam", "iliar", "word"]或其他一些子词分解，具体分解的方式取决于其训练的 BPE 合并方式
- 原始 BPE 分词器可以在以下链接找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用的是来自 OpenAI 的开源 tiktoken 库的 BPE 分词器，这个分词器使用 Rust 语言实现了其核心算法以提升计算性能
- 我在 ./bytepair_encoder 中创建了一个notebook，用于比较这两种实现（在示例文本上，tiktoken 大约比其他快 5 倍）

In [32]:
# pip install tiktoken

In [33]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [35]:
tokenizer = tiktoken.get_encoding("gpt2")

In [36]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [37]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 分词器将未知词分解为子词和单个字符：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 使用滑动窗口进行数据采样

- 我们训练 LLMs 逐个生成单词，所以我们需要按这种方式准备训练数据，如下图序列中的下一个单词代表要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们希望包含输入和目标
- 我们希望模型预测下一个词，那么目标就是将输入向右移动一个位置

In [39]:
enc_sample = enc_text[50:]

In [40]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 接下来逐个来看，预测将如下所示：

In [41]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [42]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后续章节中去学习注意力机制，以及如何处理下一个词
- 到目前为止，我们已经实现了一个简单的数据加载器，它遍历输入数据集并返回输入和目标值向右偏移一位的结果

- 安装并导入 PyTorch（安装建议见附录 A）

In [43]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


- 我们使用滑动窗口方法，位置每次移动+1：:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取片段

In [44]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [46]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

让我们用批大小为 1 来测试一个上下文大小为 4 的 LLM 的 数据加载器:
- raw_text	输入的原始文本，字符串类型，比如 "Hello, how are you today?"
- batch_size=1	每次 yield 出的样本数量（通常是 token id 的张量），为 1 表示每次输出一个样本
- max_length=4	每个样本中 token 的最大长度，即序列片段长度是 4
- stride=1	滑窗步长，表示下一个样本与上一个样本之间的偏移步数
- shuffle=False	是否在切分后的所有样本中打乱顺序；False 表示按顺序生成样本

In [47]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [48]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

- 下面是一个使用步长等于上下文长度（此处：4）的示例：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

我们也可以创建批量的输出 ， 这里增加了步长，以避免批次之间出现重叠，因为更多的重叠可能会导致过拟合增加 ：

In [49]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建 token 嵌入

到目前为止，数据基本上已经处理完可以交给LLM了。最后，我们还需要通过嵌入的方式，将token嵌入到连续的向量中。一般这些嵌入层是包含在LLM中的，可以再模型训练过程中进行更新。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，输入 id 分别为 2、3、5 和 1（在分词后）：

In [50]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简化这个案例，假设我们有一个只有 6 个词的小词汇表，并且我们想要创建大小为 3 的嵌入：

In [51]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将产生一个 6x3 的权重矩阵：

In [52]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉One-Hot编码的人来说，上述嵌入层方法本质上只是实现One-Hot编码后使用全连接层中的矩阵乘法的一种更高效方式，这在补充代码 ./embedding_vs_matmul 中介绍

- 因为嵌入层只是一个更高效的实现，它等同于 one-hot 编码和矩阵乘法的方法，所以可以看作是一个可以通过反向传播进行优化的神经网络层

- 要将 id 为 3 的 token 转换为 3 维向量，我们执行以下操作：

In [53]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，以上是 embedding_layer 权重矩阵的第四行
- 要嵌入上述四个 input_ids 值，我们执行 ：

In [54]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- 您可能对比较嵌入层和普通线性层的附加内容感兴趣，可以选择阅读：../03_bonus_embedding-vs-matmul

## 2.8 编码词的位置

- 嵌入层将 ID 转换为相同的向量表示，无论它们在输入序列中的位置如何：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入与词嵌入向量结合，形成大型语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- BytePair 编码器的词汇量为 50,257：
- 假设我们想将输入标记编码为 256 维的向量表示：

In [56]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们将每个批次的标记嵌入到 256 维的向量中
- 如果我们有 8 个批次的批量大小，每个批次有 4 个标记，这将产生一个 8 x 4 x 256 的张量：

In [57]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [58]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [59]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，所以我们只需再创建一个嵌入层：

In [61]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [62]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- 要创建用于 LLM 的输入嵌入，我们只需将 token 和位置嵌入相加：

In [63]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


以下是整体的流程图：

1. 输入文本被进行分词为token
2. 分词之后的词被转换成对应的token ID
3. 每一个token转换为token嵌入向量和位置向量
4. 将嵌入向量和位置向量叠加之后生成输入向量

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结和要点

阅读 ./dataloader.ipynb 代码笔记本，这是我们本章实现的数据加载器的简洁版本，在接下来的章节中训练 GPT 模型时需要用到。

阅读 ./exercise-solutions.ipynb 获取练习答案。

如果你对学习 GPT-2 分词器如何从头实现和训练感兴趣，请阅读 Byte Pair Encoding (BPE) Tokenizer From Scratch 笔记本。